# T-matrix computation for 3D acoustic scattering 

In [2]:
import bempp.api
import numpy as np
import scipy
from numba import objmode
import numba
import math
import matplotlib.pyplot as plt

Define the wavenumber $k$, the 3D scatterer, unit sphere, the vertices on the grid and function space.

In [3]:
h = 0.3 # Size of the mesh
d = [1,0,0] # wave's travel direction
k = 5 # wavenumber

unit_sphere = bempp.api.shapes.sphere(r = 1, origin=(0,0,0), h = h)
vert_sphere = unit_sphere.vertices
space_sphere = bempp.api.function_space(unit_sphere, 'P', 1)

dom_cube = bempp.api.shapes.cube(length = 2/1.8, origin=(-1/1.8,-1/1.8,-1/1.8), h = h)
vert_cube = dom_cube.vertices
space_cube = bempp.api.function_space(dom_cube,'P',1)

Here, we define the identity ($Id$), double layer ($K$) and hypersingular boundary operators ($W$) on the boudary of the domain. We can use then to construct the LHS of the Burton-Miller formulation: $\frac{1}{2}\text{Id} - \text{K} + \frac{i}{k}\text{W}$

In [4]:
identity = bempp.api.operators.boundary.sparse.identity(space_cube, space_cube,space_cube)
dlp = bempp.api.operators.boundary.helmholtz.double_layer(space_cube, space_cube,space_cube,k)
hyp = bempp.api.operators.boundary.helmholtz.hypersingular(space_cube, space_cube,space_cube,k)

burton_miller = .5 * identity - dlp + (1j/k) * hyp

In [5]:
def normalized_spherical_harmonics(p, q, x):
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    temp = np.array(scipy.special.lpmn(abs(p),q, np.cos(polar))[0]) 
    legd_poly = temp[-1,-1]
    return np.sqrt(((2*q+1)/(4*np.pi))*(math.factorial(q-abs(p))/math.factorial(q+abs(p))))*legd_poly*np.exp(1j*p*azimuth)
# scipy.special.lpmn will return two arrays of size (p+1, q+1) containing P_{q}^{p}(z) and its derivative for all orders from 0..p and degrees from 0..q.
# We use 'temp' to store the first row of this outpout and then choose the element in the last row and last column to be value of P_{q}^{p}.

def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.jv(q+0.5, k*r)

def spherical_hankel_function(q,x):
    """Spherical hankel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.hankel1(q+0.5, k*r)

def regular_spherical_wavefunctions(p,q,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,x)*normalized_spherical_harmonics(p,q,x)

def radiating_spherical_wavefunctions(p,q,x):
    """Radiating Spherical Wavefunction"""
    return spherical_hankel_function(q,x)*normalized_spherical_harmonics(p,q,x)

In [6]:
def normalized_spherical_harmonics_dpolar(p, q, x):
    polar = np.arccos(x[2]/np.linalg.norm(x))
    azimuth = np.arctan2(x[1],x[0])
    y1 = normalized_spherical_harmonics(p, q, x)
    y2 = normalized_spherical_harmonics(p + 1, q, x)
    return (abs(p) / np.tan(polar)) * y1 + np.sqrt((q - abs(p)) * (q + abs(p) + 1)) * np.exp(-1j * azimuth) * y2

def normalized_spherical_harmonics_dazimuth(p, q, x):
    return 1j * p * normalized_spherical_harmonics(p, q, x)

def spherical_bessel_function_dr(q,x):
    r = np.linalg.norm(x)
    return k*(spherical_bessel_function(q-1,x) - ((q+1)/(k*r))*spherical_bessel_function(q,x))

In [7]:
p = 1
q = 2

In [8]:
@bempp.api.complex_callable
def dirichlet_fun(x,n,domain_index,result):
        with objmode():
            result[0] = regular_spherical_wavefunctions(p,q,x)
            
@bempp.api.complex_callable
def neumann_fun(x, n, domain_index, result):
    with objmode():
        r = np.linalg.norm(x)
        azimuth = np.arctan2(x[1],x[0])
        polar = np.arccos(x[2]/r)
        r_n = np.linalg.norm(n)
        
        grad_regular_sph_wf = [spherical_bessel_function_dr(q,x)*normalized_spherical_harmonics(p,q,x),
                              (1/r)*normalized_spherical_harmonics_dpolar(p,q,x)*spherical_bessel_function(q,x),
                              (1/(r*np.sin(polar)))*normalized_spherical_harmonics_dazimuth(p,q,x)*spherical_bessel_function(q,x)]
        normal_in_sph_coord = [r_n, np.arccos(n[2]/r_n), np.arctan2(n[1],n[0])]
        
        result[0] = np.inner(grad_regular_sph_wf, normal_in_sph_coord)

In [11]:
dirichlet_grid_fun = bempp.api.GridFunction(space_cube, fun = dirichlet_fun)
neumann_grid_fun = bempp.api.GridFunction(space_cube, fun = neumann_fun)
rhs_fun = dirichlet_grid_fun + (1j/k)*neumann_grid_fun
total_field, info, it_count = bempp.api.linalg.gmres(burton_miller,rhs_fun,use_strong_form=True, return_iteration_count=True)

In [12]:
def far_field(x):
    dlp_far_field = bempp.api.operators.far_field.helmholtz.double_layer(space_cube,x,k)
    return dlp_far_field*total_field

In [13]:
far_field(vert_sphere[:,0:1]) # If the point has the shape (3,1), then it will give us the solution

array([[-0.06615615-0.00018958j]])

In [14]:
far_field(vert_sphere[:,0]) # in this case, the point has shape (3,), it will report us an error, /
                            # so will do when we contruct the grid function of the far field on the sphere

IndexError: tuple index out of range

In [15]:
@bempp.api.complex_callable
def far_field_fun(x,n,domain_index,result):
    with objmode():
        result[0] = far_field(x)

In [16]:
far_field_grid = bempp.api.GridFunction(space_sphere,fun = far_field_fun)
far_field_coeff = far_field_grid.coefficients

IndexError: tuple index out of range

In [17]:
@bempp.api.complex_callable
def sph_harm(x,n,domain_index,result):
    with objmode():
        result[0] = normalized_spherical_harmonics(p,q,x)

In [18]:
sph_harm_grid = bempp.api.GridFunction(space_sphere,fun = sph_harm) 
sph_harm_coeff = sph_harm_grid.coefficients

In [19]:
mass_mat = bempp.api.operators.boundary.sparse.identity(space_sphere,space_sphere,space_sphere).weak_form().A

In [20]:
# After we have the coefficients of the far field grid function and coefficients of the spherical harmonics grid function on the unit sphere, we can compute 
# the inner product to obtain the elements inside of the scattering matrix

### Test the T-matrix 

We set the incident wave $u^{i}$ as a plain wave $e^{ikx}$ and the first task is to expand it with respect to the basis $\tilde{e_{q}^{p}}$ and find the coefficients.

In [ ]:
# need to compute coff_inc here
mass_mat = bempp.api.operators.boundary.sparse.identity(space,space,space).weak_form().A

@bempp.api.complex_callable
def inc_wave_fun(x,n,domain_index, result):
    result[0] = np.exp(1j*k*x[0])

inc_wave_grid_fun = bempp.api.GridFunction(space, fun = inc_wave_fun)

inc_wave_grid_coeff = inc_wave_grid_fun.coefficients

dirichlet_grid_coeff = []

for q in range(1,deg+1):
    for p in range(-q+1,q):
        @bempp.api.complex_callable
        def dirichlet_fun(x,n,domain_index,result):
            with objmode():
                result[0] = regular_spherical_wavefunctions(p,q,k,x)
        
        dirichlet_grid_fun = bempp.api.GridFunction(space, fun = dirichlet_fun)

        dirichlet_grid_coeff.append(dirichlet_grid_fun.coefficients)

In [ ]:
coeff_inc = np.zeros(deg**2, dtype = complex)
for i in range(deg**2):
    coeff_inc[i] = inc_wave_grid_coeff.T @ mass_mat @ dirichlet_grid_coeff[i]

Use this incident wave to compute its corresponding scattered field's far field pattern

In [ ]:
@bempp.api.complex_callable
def dirichlet_fun_exact(x,n,domain_index,result):
    result[0] = np.exp(1j * k * x[0])
            
@bempp.api.complex_callable
def neumann_fun_exact(x, n, domain_index, result):
    result[0] = 1j * k * n[0] * np.exp(1j * k * x[0])

In [ ]:
dirichlet_grid_fun_exact = bempp.api.GridFunction(space, fun = dirichlet_fun_exact)
neumann_grid_fun_exact = bempp.api.GridFunction(space, fun = neumann_fun_exact)

rhs_fun_exact = dirichlet_grid_fun_exact + (1j/k)*neumann_grid_fun_exact

In [ ]:
total_field_exact, info_exact, it_count_exact = bempp.api.linalg.gmres(burton_miller,rhs_fun_exact,use_strong_form=True, return_iteration_count=True)
dlp_far_field_exact = bempp.api.operators.far_field.helmholtz.double_layer(space,vert,k)
far_field_exact = dlp_far_field_exact * total_field_exact

Compute the scattered field's coefficients using the T-matrix above

In [ ]:
coeff_sca = T_matrix @ coeff_inc

Use the formular $$u^{s}(\boldsymbol{x}) = \sum_{q = 0}^{\infty}\sum_{|p|\leq q}a_{q}^{p}e_{q}^{p}(\boldsymbol{x})$$ to construct scattered field 

In [ ]:
def sca_wave_appro(x):
    wf_list = []
    for q in range(1,deg+1):
        for p in range(-q+1,q):
            wf_list.append((1/k)*(-1j)**(q+1)*normalized_spherical_harmonics(p,q,x))
    sca_wave_appro = np.inner(coeff_sca,wf_list)
    return sca_wave_appro

In [ ]:
far_field_approx_real_part = []
for i in range(vert.shape[1]):
    far_field_approx_real_part.append(sca_wave_appro(vert[:,i]).real)

In [ ]:
%matplotlib inline
x_axis = np.linspace(1,vert.shape[1],vert.shape[1])
plt.plot(x_axis,far_field_approx_real_part)
plt.plot(x_axis,np.squeeze(np.asarray(far_field_exact.real)))